# function to extract data and create a dataframe

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

import xarray as xr
import pandas as pd 
import polars as pl
from config import * 

file_path = os.path.join(NETCDF_FILE, "01012000_rrs_mediterranean_cmems_monthly.nc")
data = xr.open_dataset(file_path, decode_times=True)

csv_path = os.path.join(CSV_DIR, "jan.csv")
df_csv = data.to_dataframe().reset_index()
df_csv.to_csv(csv_path, index=False)

df_rrs = data.to_dataframe().dropna().reset_index()
df_lat_lon = data.to_dataframe().reset_index()

## there are NaNs values and the time is 2002-12-31

In [2]:
df_rrs['time'] = pd.Timestamp("2000-01-15")
df_lat_lon['time'] = pd.Timestamp("2000-01-15")
df_pl = pl.from_pandas(df_rrs)
df = df_lat_lon
print(df.columns)

Index(['time', 'lat', 'lon', 'RRS412', 'RRS443', 'RRS490', 'RRS510', 'RRS555',
       'RRS670'],
      dtype='object')


In [4]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import numpy as np
from matplotlib.colors import LogNorm

fig = plt.figure(figsize=(18, 12))

for idx, wl in enumerate(WAVELENGTH_DIM): 
    ax = plt.subplot(2, 3, idx + 1, projection=ccrs.PlateCarree())
    scatter = ax.scatter(df['lon'], df['lat'], c=df[f'RRS{wl}'], s=1, cmap='viridis',
                 alpha=0.8, transform=ccrs.PlateCarree(), vmin=df[f'RRS{wl}'].quantile(0.01), vmax=df[f'RRS{wl}'].quantile(0.99))
    cbar = plt.colorbar(scatter, ax=ax, orientation='horizontal', 
                       pad=0.05, shrink=0.8)
    cbar.set_label(f'{wl} Reflectance', fontsize=12)
    cbar.ax.tick_params(labelsize=8)
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines(resolution='10m', linewidth=0.5, color='black')
    ax.add_feature(cfeature.LAND, facecolor='lightgray', alpha=0.3)
    ax.set_title(f'RRS {wl} nm', fontsize=11, fontweight='bold')

fig.suptitle(f'Mediterranean Sea Remote Sensing Reflectance, January 2000', 
                fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


/home/andrea/ogs/mediterranean_analysis/venv/lib/python3.11/site-packages/cartopy/io/__init__.py:242: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)
/home/andrea/ogs/mediterranean_analysis/venv/lib/python3.11/site-packages/cartopy/io/__init__.py:242: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/10m_physical/ne_10m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


KeyboardInterrupt: 